In [5]:
import pandas as pd
import numpy as np
import math

# Entropy calculation
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts=True)
    total = np.sum(counts)
    ent = -np.sum([(count/total) * math.log2(count/total) for count in counts if count > 0])
    return ent

# Attribute entropy
def attr_entropy(data, attr, target, threshold=None):
    if threshold is not None:  # numerical attribute
        left_split = data[data[attr] < threshold]
        right_split = data[data[attr] >= threshold]
        total = len(data)
        return (len(left_split)/total) * entropy(left_split[target]) + \
               (len(right_split)/total) * entropy(right_split[target])
    else:  # categorical attribute
        values, counts = np.unique(data[attr], return_counts=True)
        total = np.sum(counts)
        return np.sum([(counts[i]/total) * entropy(data[data[attr] == values[i]][target])
                       for i in range(len(values))])

# Info gain
def info_gain(data, attr, target, threshold=None):
    total_ent = entropy(data[target])
    attr_ent = attr_entropy(data, attr, target, threshold)
    return total_ent - attr_ent

# Split info
def split_info(data, attr, threshold=None):
    if threshold is not None:  # numerical
        left_split = data[data[attr] < threshold]
        right_split = data[data[attr] >= threshold]
        total = len(data)
        splits = [len(left_split), len(right_split)]
    else:  # categorical
        _, counts = np.unique(data[attr], return_counts=True)
        splits = counts
        total = np.sum(counts)
    return -np.sum([(c/total) * math.log2(c/total) for c in splits if c > 0])

# Gain ratio (with printing)
def gain_ratio(data, attr, target, verbose=False):
    if np.issubdtype(data[attr].dtype, np.number):  # numerical feature
        values = np.sort(data[attr].unique())
        if len(values) == 1:  # all values same
            return 0, None
        thresholds = [(values[i] + values[i+1])/2 for i in range(len(values)-1)]
        best_gain_ratio = -1
        best_threshold = None
        for t in thresholds:
            ig = info_gain(data, attr, target, threshold=t)
            si = split_info(data, attr, threshold=t)
            gr = ig/si if si != 0 else 0
            if verbose:
                print(f"[NUM] {attr} | Threshold={t:.3f} | InfoGain={ig:.4f} | SplitInfo={si:.4f} | GainRatio={gr:.4f}")
            if gr > best_gain_ratio:
                best_gain_ratio = gr
                best_threshold = t
        return best_gain_ratio, best_threshold
    else:  # categorical feature
        ig = info_gain(data, attr, target)
        si = split_info(data, attr)
        gr = ig/si if si != 0 else 0
        if verbose:
            print(f"{attr} | InfoGain={ig:.4f} | SplitInfo={si:.4f} | GainRatio={gr:.4f}")
        return gr, None

# Recursive C4.5 algorithm
def C45(data, orig_data, features, target, parent_class=None, depth=0):
    prefix = "   " * depth
    if len(np.unique(data[target])) == 1:
        return np.unique(data[target])[0]
    elif len(data) == 0:
        return np.unique(orig_data[target])[np.argmax(np.unique(orig_data[target], return_counts=True)[1])]
    elif len(features) == 0:
        return parent_class
    else:
        parent_class = np.unique(data[target])[np.argmax(np.unique(data[target], return_counts=True)[1])]
        
        print(f"\n{prefix}Entropy(Target) = {entropy(data[target]):.4f}")
        print(f"{prefix}Checking features...")

        # Find best feature
        gains = [gain_ratio(data, f, target, verbose=True) for f in features]
        best_feature_index = np.argmax([g[0] for g in gains])
        best_feature = features[best_feature_index]
        best_gain, best_threshold = gains[best_feature_index]

        print(f"{prefix}--> Best Feature = {best_feature}, Best GainRatio = {best_gain:.4f}, Threshold = {best_threshold}")

        if best_gain == 0:
            return parent_class

        tree = {best_feature: {}}

        if best_threshold is not None:  # numerical split
            left_split = data[data[best_feature] < best_threshold]
            right_split = data[data[best_feature] >= best_threshold]
            sub_feat = [f for f in features if f != best_feature]
            tree[best_feature][f"< {best_threshold}"] = C45(left_split, orig_data, sub_feat, target, parent_class, depth+1)
            tree[best_feature][f">= {best_threshold}"] = C45(right_split, orig_data, sub_feat, target, parent_class, depth+1)
        else:  # categorical split
            for value in np.unique(data[best_feature]):
                sub_data = data[data[best_feature] == value]
                sub_feat = [f for f in features if f != best_feature]
                tree[best_feature][value] = C45(sub_data, orig_data, sub_feat, target, parent_class, depth+1)

        return tree


# ==== Usage ====
df = pd.read_csv("diabetes.csv")
target_attr = 'Outcome'
features = [f for f in df.columns if f != target_attr]

decision_tree = C45(df, df, features, target_attr)

import pprint
print("\nFinal Decision Tree:")
pprint.pprint(decision_tree)



Entropy(Target) = 0.9521
Checking features...
[NUM] Pregnancies | Threshold=0.500 | InfoGain=0.0061 | SplitInfo=0.5016 | GainRatio=0.0122
[NUM] Pregnancies | Threshold=1.500 | InfoGain=0.0024 | SplitInfo=0.8505 | GainRatio=0.0028
[NUM] Pregnancies | Threshold=2.500 | InfoGain=0.0254 | SplitInfo=0.9721 | GainRatio=0.0261
[NUM] Pregnancies | Threshold=3.500 | InfoGain=0.0275 | SplitInfo=0.9985 | GainRatio=0.0275
[NUM] Pregnancies | Threshold=4.500 | InfoGain=0.0442 | SplitInfo=0.9824 | GainRatio=0.0450
[NUM] Pregnancies | Threshold=5.500 | InfoGain=0.0654 | SplitInfo=0.9006 | GainRatio=0.0726
[NUM] Pregnancies | Threshold=6.500 | InfoGain=0.0690 | SplitInfo=0.8362 | GainRatio=0.0825
[NUM] Pregnancies | Threshold=7.500 | InfoGain=0.0516 | SplitInfo=0.6481 | GainRatio=0.0797
[NUM] Pregnancies | Threshold=8.500 | InfoGain=0.0214 | SplitInfo=0.4863 | GainRatio=0.0440
[NUM] Pregnancies | Threshold=9.500 | InfoGain=0.0089 | SplitInfo=0.3672 | GainRatio=0.0244
[NUM] Pregnancies | Threshold=10.

[NUM] Insulin | Threshold=158.000 | InfoGain=0.0315 | SplitInfo=0.5990 | GainRatio=0.0525
[NUM] Insulin | Threshold=164.000 | InfoGain=0.0270 | SplitInfo=0.5861 | GainRatio=0.0461
[NUM] Insulin | Threshold=171.500 | InfoGain=0.0188 | SplitInfo=0.5592 | GainRatio=0.0337
[NUM] Insulin | Threshold=175.500 | InfoGain=0.0118 | SplitInfo=0.5311 | GainRatio=0.0222
[NUM] Insulin | Threshold=184.000 | InfoGain=0.0145 | SplitInfo=0.5165 | GainRatio=0.0282
[NUM] Insulin | Threshold=198.000 | InfoGain=0.0177 | SplitInfo=0.5016 | GainRatio=0.0353
[NUM] Insulin | Threshold=205.500 | InfoGain=0.0214 | SplitInfo=0.4863 | GainRatio=0.0440
[NUM] Insulin | Threshold=208.500 | InfoGain=0.0173 | SplitInfo=0.4706 | GainRatio=0.0367
[NUM] Insulin | Threshold=215.000 | InfoGain=0.0135 | SplitInfo=0.4545 | GainRatio=0.0296
[NUM] Insulin | Threshold=222.500 | InfoGain=0.0100 | SplitInfo=0.4380 | GainRatio=0.0228
[NUM] Insulin | Threshold=226.500 | InfoGain=0.0069 | SplitInfo=0.4210 | GainRatio=0.0164
[NUM] Insu

[NUM] DiabetesPedigreeFunction | Threshold=0.325 | InfoGain=0.0261 | SplitInfo=0.9867 | GainRatio=0.0264
[NUM] DiabetesPedigreeFunction | Threshold=0.330 | InfoGain=0.0224 | SplitInfo=0.9886 | GainRatio=0.0227
[NUM] DiabetesPedigreeFunction | Threshold=0.335 | InfoGain=0.0244 | SplitInfo=0.9903 | GainRatio=0.0247
[NUM] DiabetesPedigreeFunction | Threshold=0.337 | InfoGain=0.0265 | SplitInfo=0.9920 | GainRatio=0.0267
[NUM] DiabetesPedigreeFunction | Threshold=0.340 | InfoGain=0.0249 | SplitInfo=0.9947 | GainRatio=0.0250
[NUM] DiabetesPedigreeFunction | Threshold=0.343 | InfoGain=0.0270 | SplitInfo=0.9959 | GainRatio=0.0271
[NUM] DiabetesPedigreeFunction | Threshold=0.347 | InfoGain=0.0233 | SplitInfo=0.9969 | GainRatio=0.0234
[NUM] DiabetesPedigreeFunction | Threshold=0.353 | InfoGain=0.0254 | SplitInfo=0.9978 | GainRatio=0.0254
[NUM] DiabetesPedigreeFunction | Threshold=0.358 | InfoGain=0.0239 | SplitInfo=0.9991 | GainRatio=0.0239
[NUM] DiabetesPedigreeFunction | Threshold=0.364 | Info

[NUM] Insulin | Threshold=54.500 | InfoGain=0.0013 | SplitInfo=0.9448 | GainRatio=0.0014
[NUM] Insulin | Threshold=56.500 | InfoGain=0.0019 | SplitInfo=0.9399 | GainRatio=0.0020
[NUM] Insulin | Threshold=60.500 | InfoGain=0.0007 | SplitInfo=0.9348 | GainRatio=0.0008
[NUM] Insulin | Threshold=63.500 | InfoGain=0.0011 | SplitInfo=0.9296 | GainRatio=0.0012
[NUM] Insulin | Threshold=67.500 | InfoGain=0.0022 | SplitInfo=0.9183 | GainRatio=0.0024
[NUM] Insulin | Threshold=73.500 | InfoGain=0.0036 | SplitInfo=0.9062 | GainRatio=0.0040
[NUM] Insulin | Threshold=77.000 | InfoGain=0.0045 | SplitInfo=0.8997 | GainRatio=0.0050
[NUM] Insulin | Threshold=80.000 | InfoGain=0.0055 | SplitInfo=0.8931 | GainRatio=0.0061
[NUM] Insulin | Threshold=82.500 | InfoGain=0.0065 | SplitInfo=0.8862 | GainRatio=0.0074
[NUM] Insulin | Threshold=85.500 | InfoGain=0.0077 | SplitInfo=0.8791 | GainRatio=0.0088
[NUM] Insulin | Threshold=89.000 | InfoGain=0.0051 | SplitInfo=0.8718 | GainRatio=0.0058
[NUM] Insulin | Thres

[NUM] DiabetesPedigreeFunction | Threshold=0.364 | InfoGain=0.0025 | SplitInfo=0.9980 | GainRatio=0.0025
[NUM] DiabetesPedigreeFunction | Threshold=0.369 | InfoGain=0.0032 | SplitInfo=0.9970 | GainRatio=0.0032
[NUM] DiabetesPedigreeFunction | Threshold=0.372 | InfoGain=0.0047 | SplitInfo=0.9944 | GainRatio=0.0048
[NUM] DiabetesPedigreeFunction | Threshold=0.378 | InfoGain=0.0056 | SplitInfo=0.9929 | GainRatio=0.0057
[NUM] DiabetesPedigreeFunction | Threshold=0.385 | InfoGain=0.0066 | SplitInfo=0.9911 | GainRatio=0.0067
[NUM] DiabetesPedigreeFunction | Threshold=0.390 | InfoGain=0.0077 | SplitInfo=0.9891 | GainRatio=0.0078
[NUM] DiabetesPedigreeFunction | Threshold=0.394 | InfoGain=0.0101 | SplitInfo=0.9845 | GainRatio=0.0102
[NUM] DiabetesPedigreeFunction | Threshold=0.398 | InfoGain=0.0114 | SplitInfo=0.9819 | GainRatio=0.0116
[NUM] DiabetesPedigreeFunction | Threshold=0.401 | InfoGain=0.0128 | SplitInfo=0.9792 | GainRatio=0.0131
[NUM] DiabetesPedigreeFunction | Threshold=0.404 | Info

[NUM] Insulin | Threshold=277.000 | InfoGain=0.0131 | SplitInfo=0.1959 | GainRatio=0.0667
[NUM] Insulin | Threshold=284.500 | InfoGain=0.0104 | SplitInfo=0.1646 | GainRatio=0.0633
[NUM] Insulin | Threshold=305.000 | InfoGain=0.0078 | SplitInfo=0.1311 | GainRatio=0.0594
[NUM] Insulin | Threshold=333.500 | InfoGain=0.0052 | SplitInfo=0.0945 | GainRatio=0.0547
[NUM] Insulin | Threshold=413.500 | InfoGain=0.0026 | SplitInfo=0.0534 | GainRatio=0.0483
[NUM] DiabetesPedigreeFunction | Threshold=0.085 | InfoGain=0.0026 | SplitInfo=0.0534 | GainRatio=0.0483
[NUM] DiabetesPedigreeFunction | Threshold=0.086 | InfoGain=0.0052 | SplitInfo=0.0945 | GainRatio=0.0547
[NUM] DiabetesPedigreeFunction | Threshold=0.092 | InfoGain=0.0078 | SplitInfo=0.1311 | GainRatio=0.0594
[NUM] DiabetesPedigreeFunction | Threshold=0.099 | InfoGain=0.0104 | SplitInfo=0.1646 | GainRatio=0.0633
[NUM] DiabetesPedigreeFunction | Threshold=0.116 | InfoGain=0.0131 | SplitInfo=0.1959 | GainRatio=0.0667
[NUM] DiabetesPedigreeFun

[NUM] BloodPressure | Threshold=71.000 | InfoGain=0.0159 | SplitInfo=0.9999 | GainRatio=0.0159
[NUM] BloodPressure | Threshold=73.000 | InfoGain=0.0071 | SplitInfo=0.9931 | GainRatio=0.0071
[NUM] BloodPressure | Threshold=74.500 | InfoGain=0.0085 | SplitInfo=0.9522 | GainRatio=0.0089
[NUM] BloodPressure | Threshold=75.500 | InfoGain=0.0124 | SplitInfo=0.9373 | GainRatio=0.0132
[NUM] BloodPressure | Threshold=77.000 | InfoGain=0.0033 | SplitInfo=0.8943 | GainRatio=0.0037
[NUM] BloodPressure | Threshold=79.000 | InfoGain=0.0128 | SplitInfo=0.8300 | GainRatio=0.0154
[NUM] BloodPressure | Threshold=81.000 | InfoGain=0.0187 | SplitInfo=0.7244 | GainRatio=0.0258
[NUM] BloodPressure | Threshold=83.000 | InfoGain=0.0198 | SplitInfo=0.6308 | GainRatio=0.0314
[NUM] BloodPressure | Threshold=84.500 | InfoGain=0.0138 | SplitInfo=0.5687 | GainRatio=0.0243
[NUM] BloodPressure | Threshold=85.500 | InfoGain=0.0110 | SplitInfo=0.5349 | GainRatio=0.0206
[NUM] BloodPressure | Threshold=87.000 | InfoGain=

[NUM] DiabetesPedigreeFunction | Threshold=0.540 | InfoGain=0.0216 | SplitInfo=0.8798 | GainRatio=0.0246
[NUM] DiabetesPedigreeFunction | Threshold=0.544 | InfoGain=0.0166 | SplitInfo=0.8722 | GainRatio=0.0190
[NUM] DiabetesPedigreeFunction | Threshold=0.548 | InfoGain=0.0184 | SplitInfo=0.8643 | GainRatio=0.0213
[NUM] DiabetesPedigreeFunction | Threshold=0.557 | InfoGain=0.0138 | SplitInfo=0.8561 | GainRatio=0.0161
[NUM] DiabetesPedigreeFunction | Threshold=0.568 | InfoGain=0.0155 | SplitInfo=0.8477 | GainRatio=0.0183
[NUM] DiabetesPedigreeFunction | Threshold=0.577 | InfoGain=0.0174 | SplitInfo=0.8390 | GainRatio=0.0207
[NUM] DiabetesPedigreeFunction | Threshold=0.585 | InfoGain=0.0128 | SplitInfo=0.8300 | GainRatio=0.0154
[NUM] DiabetesPedigreeFunction | Threshold=0.587 | InfoGain=0.0145 | SplitInfo=0.8208 | GainRatio=0.0176
[NUM] DiabetesPedigreeFunction | Threshold=0.592 | InfoGain=0.0163 | SplitInfo=0.8113 | GainRatio=0.0201
[NUM] DiabetesPedigreeFunction | Threshold=0.612 | Info

[NUM] Age | Threshold=42.500 | InfoGain=0.0125 | SplitInfo=0.5871 | GainRatio=0.0213
[NUM] Age | Threshold=43.500 | InfoGain=0.0079 | SplitInfo=0.5709 | GainRatio=0.0138
[NUM] Age | Threshold=44.500 | InfoGain=0.0097 | SplitInfo=0.5542 | GainRatio=0.0176
[NUM] Age | Threshold=45.500 | InfoGain=0.0171 | SplitInfo=0.5012 | GainRatio=0.0342
[NUM] Age | Threshold=46.500 | InfoGain=0.0114 | SplitInfo=0.4825 | GainRatio=0.0235
[NUM] Age | Threshold=47.500 | InfoGain=0.0140 | SplitInfo=0.4631 | GainRatio=0.0302
[NUM] Age | Threshold=49.000 | InfoGain=0.0171 | SplitInfo=0.4432 | GainRatio=0.0385
[NUM] Age | Threshold=50.500 | InfoGain=0.0139 | SplitInfo=0.4013 | GainRatio=0.0347
[NUM] Age | Threshold=52.500 | InfoGain=0.0082 | SplitInfo=0.3793 | GainRatio=0.0215
[NUM] Age | Threshold=54.500 | InfoGain=0.0055 | SplitInfo=0.3328 | GainRatio=0.0165
[NUM] Age | Threshold=55.500 | InfoGain=0.0080 | SplitInfo=0.3081 | GainRatio=0.0259
[NUM] Age | Threshold=56.500 | InfoGain=0.0031 | SplitInfo=0.2825

[NUM] SkinThickness | Threshold=43.000 | InfoGain=0.0028 | SplitInfo=0.3228 | GainRatio=0.0086
[NUM] SkinThickness | Threshold=45.500 | InfoGain=0.0011 | SplitInfo=0.2515 | GainRatio=0.0045
[NUM] SkinThickness | Threshold=48.500 | InfoGain=0.0138 | SplitInfo=0.1698 | GainRatio=0.0810
[NUM] SkinThickness | Threshold=52.000 | InfoGain=0.0091 | SplitInfo=0.1231 | GainRatio=0.0741
[NUM] SkinThickness | Threshold=57.000 | InfoGain=0.0045 | SplitInfo=0.0700 | GainRatio=0.0648
                     --> Best Feature = SkinThickness, Best GainRatio = 0.0810, Threshold = 48.5

                     Entropy(Target) = 0.8113
                     Checking features...
[NUM] SkinThickness | Threshold=16.500 | InfoGain=0.3113 | SplitInfo=1.0000 | GainRatio=0.3113
[NUM] SkinThickness | Threshold=37.000 | InfoGain=0.1226 | SplitInfo=0.8113 | GainRatio=0.1511
                     --> Best Feature = SkinThickness, Best GainRatio = 0.3113, Threshold = 16.5

   Entropy(Target) = 0.3712
   Checking features...